In [1]:
import os
import subprocess
import uuid
import asyncio
import yaml
import chromadb
import requests
import base64
import zlib
import json
import argparse
import tempfile
from chromadb.config import Settings
from chromadb.utils import embedding_functions
from autogen_core.memory import MemoryContent, MemoryMimeType
from autogen_core.models import ChatCompletionClient, ModelInfo
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.teams import RoundRobinGroupChat, Swarm, SelectorGroupChat
from autogen_agentchat.conditions import TextMentionTermination, HandoffTermination, TextMentionTermination, MaxMessageTermination
from autogen_agentchat.messages import HandoffMessage
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_ext.models.ollama import OllamaChatCompletionClient
from autogen_ext.memory.chromadb import ChromaDBVectorMemory, PersistentChromaDBVectorMemoryConfig
from autogen_ext.auth.azure import AzureTokenProvider
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from azure.identity import DefaultAzureCredential
from langchain_community.document_loaders import PyMuPDFLoader, TextLoader, UnstructuredWordDocumentLoader, WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
AZURE = "C:/Master IA/TFM_BA_GPT/src/model_config_azure.yaml"
OLLAMA = "C:/Master IA/TFM_BA_GPT/src/model_config_ollama.yaml"
LMSTUDIO = "C:/Master IA/TFM_BA_GPT/src/model_config_lmstudio.yaml"

# Load model configuration and create the model client.
with open(OLLAMA, "r") as f:
    model_config = yaml.safe_load(f)
    
model_client = ChatCompletionClient.load_component(model_config)

Agent to make diagram

In [3]:
def read_txt_file(file_path: str) -> str:
    """
    Reads the content of a .txt file and returns it as a string.
    """
    try:
        with open(file_path, "r", encoding="utf-8") as f:
            return f.read()
    except Exception as e:
        return f"Error reading file: {e}"
    
def read_pdf_file(file_path: str) -> str:
    """
    Reads the content of a .pdf file and returns it as a string.
    """
    try:
        loader = PyMuPDFLoader(file_path)
        documents = loader.load()
        return documents[0].page_content if documents else ""
    except Exception as e:
        return f"Error reading file: {e}"
    
def write_mermaid_to_file(mermaid_code: str, filename: str) -> None:
    """
    Write Mermaid diagram code to a .mmd file.

    Args:
        mermaid_code (str): The Mermaid syntax/code to write.
        filename (str): The name of the file to write to. Should end with '.mmd'.
    """
    if not filename.endswith('.mmd'):
        raise ValueError("Filename must end with '.mmd'")
    
    with open(filename, 'w', encoding='utf-8') as file:
        file.write("```mermaid\n")
        file.write(mermaid_code)
        file.write("\n```")

    print(f"Mermaid diagram written to {filename}")




In [5]:
planning_agent = AssistantAgent(
    "PlanningAgent",
    description="An agent for planning tasks, this agent should be the first to engage when given a new task.",
    model_client=model_client,
    system_message="""
    You are a planning agent.
    Your job is to break down complex tasks into smaller, manageable subtasks.
    Your team members are:
    - summarizer: An agent that uses tools to read pdf files and summarizes the process information.
    - diagram_creator_agent: An agent that receives the summary of a process and creates diagrams as a code using Mermaid languagge.
    - mermaid_code_reviewer_agent: An agent that reviews the Mermaid code from the diagram_creator_agent and provides feedback.

    You only plan and delegate tasks - you do not execute them yourself.

    When assigning tasks, use this format:
    1. <agent> : <task>

    After assigning tasks, wait for all agents to finish their tasks.
    After all tasks are complete, summarize the findings and end with "TERMINATE".
    """,
    model_client_stream=True,
)

diagram_creator_agent = AssistantAgent(
    name="diagram_creator_agent",
    description="An agent that create diagrams as a code.",
    model_client=model_client,
    system_message="""
    You are a Process Diagram Creator and an expert in the Mermaid language.
    Your job is to create process diagrams as a code using the Mermaid language.
    You will receive a summary of the information provided by the summarizer agent.
    """,
    model_client_stream=True,
)

summarizer_agent = AssistantAgent(
        name="summarizer",
        description="An agent that use tools to read pdf files and summarizes the information.",
        model_client=model_client,
        tools=[read_pdf_file],
        system_message=(
            "You are an expert on making process summaries."
            "You will read a pdf file using tools and summarize the information."
            "Make sure to include all the important information."
            "You do not generate Mermaid code"
        ),
        model_client_stream=True,  # Enable model client streaming.
    )

mermaid_code_reviewer_agent = AssistantAgent(
        name="mermaid_code_reviewer_agent",
        description="An agent that reviews the Mermaid code.",
        model_client=model_client,
        tools=[write_mermaid_to_file],
        system_message=(
            "You are an expert on the Mermaid languagge."
            "Review the code generated by the diagram_creator_agent"
            "If there is feedback, provide it to the PlanningAgent."
            "else use the code with write_mermaid_to_file to save as C:/Master IA/TFM_BA_GPT/doc/mermaid.mmd and end with 'TERMINATE'."
        ),
)

In [6]:
text_mention_termination = TextMentionTermination("TERMINATE")
max_messages_termination = MaxMessageTermination(max_messages=25)
termination = text_mention_termination | max_messages_termination

In [7]:
selector_prompt = """Select an agent to perform task.

{roles}

Current conversation context:
{history}

Read the above conversation, then select an agent from {participants} to perform the next task.
Make sure the planner agent has assigned tasks before other agents start working.
Only select one agent.
"""

In [8]:
team = SelectorGroupChat(
    [planning_agent, summarizer_agent, diagram_creator_agent, mermaid_code_reviewer_agent],
    model_client=model_client,
    termination_condition=termination,
    selector_prompt=selector_prompt,
    allow_repeated_speaker=True,  # Allow an agent to speak multiple turns in a row.
)

In [9]:
task = "Read the PDF in file_path:'C:/Master IA/TFM_BA_GPT/doc/Sales SOP for acquiring new clients.pdf' file and create a diagram as a code for that process using Mermaid language"

In [10]:
await Console(team.run_stream(task=task))

---------- user ----------
Read the PDF in file_path:'C:/Master IA/TFM_BA_GPT/doc/Sales SOP for acquiring new clients.pdf' file and create a diagram as a code for that process using Mermaid language
---------- PlanningAgent ----------
1. summarizer : Read the PDF located at 'C:/Master IA/TFM_BA_GPT/doc/Sales SOP for acquiring new clients.pdf' and provide a summary of the sales process described within.
2. diagram_creator_agent : Once you receive the summary from the summarizer, create a diagram as a code using Mermaid language that represents the sales process of acquiring new clients.
3. mermaid_code_reviewer_agent : After the diagram is created by the diagram_creator_agent, review the Mermaid code and provide feedback on its accuracy and completeness.


Error processing publish message for SelectorGroupChatManager_39f6333a-e30a-4d0d-8a5a-c043b5b1e745/39f6333a-e30a-4d0d-8a5a-c043b5b1e745
Traceback (most recent call last):
  File "c:\Master IA\TFM_BA_GPT\venv\Lib\site-packages\autogen_core\_single_threaded_agent_runtime.py", line 533, in _on_message
    return await agent.on_message(
           ^^^^^^^^^^^^^^^^^^^^^^^
    ...<2 lines>...
    )
    ^
  File "c:\Master IA\TFM_BA_GPT\venv\Lib\site-packages\autogen_core\_base_agent.py", line 113, in on_message
    return await self.on_message_impl(message, ctx)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Master IA\TFM_BA_GPT\venv\Lib\site-packages\autogen_agentchat\teams\_group_chat\_sequential_routed_agent.py", line 67, in on_message_impl
    return await super().on_message_impl(message, ctx)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Master IA\TFM_BA_GPT\venv\Lib\site-packages\autogen_core\_routed_agent.py", line 485, in on_message_impl
    return

RuntimeError: ResponseError: <html>
<head><title>502 Bad Gateway</title></head>
<body>
<center><h1>502 Bad Gateway</h1></center>
<hr><center>nginx</center>
</body>
</html>
 (status code: 502)
Traceback:
Traceback (most recent call last):

  File "c:\Master IA\TFM_BA_GPT\venv\Lib\site-packages\autogen_agentchat\teams\_group_chat\_base_group_chat_manager.py", line 189, in handle_agent_response
    speaker_name = await speaker_name_future
                   ^^^^^^^^^^^^^^^^^^^^^^^^^

  File "c:\Master IA\TFM_BA_GPT\venv\Lib\site-packages\autogen_agentchat\teams\_group_chat\_selector_group_chat.py", line 174, in select_speaker
    agent_name = await self._select_speaker(roles, participants, history, self._max_selector_attempts)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

  File "c:\Master IA\TFM_BA_GPT\venv\Lib\site-packages\autogen_agentchat\teams\_group_chat\_selector_group_chat.py", line 195, in _select_speaker
    respo